# College Football Win Total Cleaning from Web Scraping 

The purpose of this project is to convert the Vegas win totals for the upcoming College Football 2023 into an easy to read hiearachy by converting the vig for the over number and recombining that with the original win total. We will scrape this data from DraftKings and then clean it prior to processing in a pandas DataFrame. This project also has a Quality Assurance section at the end for those that are interested. 

In [1]:
# Use BeautifulSoup to get the Win Total data

import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

url = 'https://dknation.draftkings.com/2023/5/16/23725731/ncaa-football-win-totals-2023-over-under-alabama-georgia-usc-texas-lsu-sec-schedule'
page = requests.get(url)
  
soup = BeautifulSoup(page.text, 'html.parser')

/Users/alecnigh/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (3.0.4)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# Let's look at our soup

soup


<!DOCTYPE html>

<html lang="en">
<head>
<title>College football win totals 2023: Over under wins for every team in FBS; Georgia, Alabama, USC, more - DraftKings Network</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="DraftKings Network" name="apple-mobile-web-app-title">
<meta content="Collin Sherwin" property="author">
<meta content="2023-05-16T10:44:07-07:00" property="article:published_time">
<meta content="2023-05-16T11:06:25-07:00" property="article:modified_time"/>
<link href="https://use.typekit.net" rel="preconnect"/>
<meta data-cdata='{"version":"c21d9ae195b0d525cd96fab205882683","fonts_catalog":[],"font_stylesheets":["https://fonts.googleapis.com/css2?family=Montserrat:ital,wght@0,400;0,600;0,700;0,900;1,400&amp;display=swap"],"font_tracker_stylesheets":[],"typekit_ids":[],"headline_balance

In [5]:
# Separate our headers

headers = soup.findAll('th')[:3]
column_headers = [i.getText() for i in headers]
column_headers

['Team', 'Over', 'Under']

In [6]:
# Getting the column headers for our code

content = soup.findAll('tbody')[0]
content = [i.getText()[0:] for i in content.findAll('td')]
content

['Air Force',
 '8.5 +125',
 '8.5 −150',
 'Akron',
 '3.5 −120',
 '3.5 −105',
 'Alabama',
 '10.5 +135',
 '10.5 −155',
 'Appalachian State',
 '6.5 −110',
 '6.5 −115',
 'Arizona',
 '5 −120',
 '5 +100',
 'Arizona State',
 '5 +120',
 '5 −140',
 'Arkansas',
 '7 +100',
 '7 −120',
 'Arkansas State',
 '4.5 −145',
 '4.5 +120',
 'Army',
 '6 +105',
 '6 −130',
 'Auburn',
 '6.5 −135',
 '6.5 +115',
 'Ball State',
 '4.5 −120',
 '4.5 −105',
 'Baylor',
 '7.5 +135',
 '7.5 −160',
 'Boise State',
 '9 +140',
 '9 −170',
 'Boston College',
 '5.5 −120',
 '5.5 +100',
 'Bowling Green',
 '5 +105',
 '5 −130',
 'BYU',
 '5.5 −120',
 '5.5 +100',
 'California',
 '5 +110',
 '5 −130',
 'Central Michigan',
 '5.5 −110',
 '5.5 −115',
 'Charlotte',
 '2.5 −120',
 '2.5 −105',
 'Cincinnati',
 '5.5 +110',
 '5.5 −130',
 'Clemson',
 '9.5 −140',
 '9.5 +120',
 'Coastal Carolina',
 '8 −120',
 '8 −105',
 'Colorado',
 '3 −105',
 '3 −115',
 'Colorado State',
 '4.5 −115',
 '4.5 −110',
 'Duke',
 '6.5 +115',
 '6.5 −135',
 'East Carolina',


Next, we need to transform this data into separate lists using the pattern of 'Team', 'Over', 'Under' in a for loop using the order of our content list to grab every third item from different starting points.

In [7]:
teams = []
Over = []
Under = []
for x in content[0::3]:
    teams.append(x)
for y in content[1::3]:
    Over.append(y)
for z in content[2::3]:
    Under.append(z)

Always good to double check and ensure all our data is there and even.

In [8]:
print("UNDER :", len(Under), 
      "OVER :", len(Over), 
      "TEAMS :", len(teams))

UNDER : 127 OVER : 127 TEAMS : 127


So there should be 133 teams in total, which means we're missing some (this is the QA part of the problem outlined above). I'll circle back to this issue once we have the dataframe together.

For now, let's transform the data using a dataframe.

In [38]:
body = {'Team': teams,
        'Over': Over,
        'Under': Under}

df = pd.DataFrame(columns = column_headers, data = body)
df.head()

,Team,Over,Under
0,Air Force,8.5 +125,8.5 −150
1,Akron,3.5 −120,3.5 −105
2,Alabama,10.5 +135,10.5 −155
3,Appalachian State,6.5 −110,6.5 −115
4,Arizona,5 −120,5 +100


Ok the data looks good, but the win totals combined with the different odds is causing a little confusion. I would prefer an apples to apples comparision so we will need to centralize our data around a similar odds -- let's transform our data to display +100 odds or 50% implied probability for each team. This way, if the odds are plus odds, like +125, they would be less likely than 50% to go over their total. And if odds are minus like -120, they would be more than 50% chance of getting their over. Thus, their total would be higher. 

Make sense? Let's manipulate some data.

The following code splits out 'Over' column to separate the win totals from the vig number at the end of the column. We then clean the newly created column and convert to the appropriate datatype -- float.

In [39]:

# Split the "Over" column by space
df[['Over_Value', 'Over_Odds']] = df['Over'].str.split(' ', 1, expand=True)

# Split the "Under" column by space
df[['Under_Value', 'Under_Odds']] = df['Under'].str.split(' ', 1, expand=True)


In [40]:
df.head()

,Team,Over,Under,Over_Value,Over_Odds,Under_Value,Under_Odds
0,Air Force,8.5 +125,8.5 −150,8.5,+125,8.5,−150
1,Akron,3.5 −120,3.5 −105,3.5,−120,3.5,−105
2,Alabama,10.5 +135,10.5 −155,10.5,+135,10.5,−155
3,Appalachian State,6.5 −110,6.5 −115,6.5,−110,6.5,−115
4,Arizona,5 −120,5 +100,5,−120,5,+100


In [41]:
# Replace Unicode minus sign with hyphen-minus
df['Over_Odds'] = df['Over_Odds'].str.replace('−', '-')
df['Under_Odds'] = df['Under_Odds'].str.replace('−', '-')

# A function to split the values and store in separate columns
def split_values(row, column):
    value = row[column]
    if value.startswith('-'):
        return value[1:], '-'
    elif value.startswith('+'):
        return value[1:], '+'
    else:
        return value, ''

# Split the 'Over_Odds' column
df['Over_Value_Split'], df['Over_Sign'] = zip(*df.apply(lambda row: split_values(row, 'Over_Odds'), axis=1))

# Split the 'Under_Odds' column
df['Under_Value_Split'], df['Under_Sign'] = zip(*df.apply(lambda row: split_values(row, 'Under_Odds'), axis=1))

# Remove leading white space from the split values
df['Over_Value_Split'] = df['Over_Value_Split'].str.strip()
df['Under_Value_Split'] = df['Under_Value_Split'].str.strip()

# Convert the split values to the appropriate data types
df['Over_Value_Split'] = df['Over_Value_Split'].astype(float)
df['Under_Value_Split'] = df['Under_Value_Split'].astype(float)

# Display the updated dataframe
print(df)

                  Team       Over      Under Over_Value Over_Odds Under_Value  \
0            Air Force   8.5 +125   8.5 −150        8.5      +125         8.5   
1                Akron   3.5 −120   3.5 −105        3.5      -120         3.5   
2              Alabama  10.5 +135  10.5 −155       10.5      +135        10.5   
3    Appalachian State   6.5 −110   6.5 −115        6.5      -110         6.5   
4              Arizona     5 −120     5 +100          5      -120           5   
..                 ...        ...        ...        ...       ...         ...   
122      West Virginia   4.5 −140   4.5 +120        4.5      -140         4.5   
123   Western Kentucky   8.5 +125   8.5 −150        8.5      +125         8.5   
124   Western Michigan   3.5 +110   3.5 −135        3.5      +110         3.5   
125          Wisconsin     9 +125     9 −145          9      +125           9   
126            Wyoming   6.5 −125   6.5 +100        6.5      -125         6.5   

    Under_Odds  Over_Value_

In [42]:
df.head()

,Team,Over,Under,Over_Value,Over_Odds,Under_Value,Under_Odds,Over_Value_Split,Over_Sign,Under_Value_Split,Under_Sign
0,Air Force,8.5 +125,8.5 −150,8.5,+125,8.5,-150,125.0,+,150.0,-
1,Akron,3.5 −120,3.5 −105,3.5,-120,3.5,-105,120.0,-,105.0,-
2,Alabama,10.5 +135,10.5 −155,10.5,+135,10.5,-155,135.0,+,155.0,-
3,Appalachian State,6.5 −110,6.5 −115,6.5,-110,6.5,-115,110.0,-,115.0,-
4,Arizona,5 −120,5 +100,5,-120,5,+100,120.0,-,100.0,+


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Team               127 non-null    object 
 1   Over               127 non-null    object 
 2   Under              127 non-null    object 
 3   Over_Value         127 non-null    object 
 4   Over_Odds          127 non-null    object 
 5   Under_Value        127 non-null    object 
 6   Under_Odds         127 non-null    object 
 7   Over_Value_Split   127 non-null    float64
 8   Over_Sign          127 non-null    object 
 9   Under_Value_Split  127 non-null    float64
 10  Under_Sign         127 non-null    object 
dtypes: float64(2), object(9)
memory usage: 11.0+ KB


In [44]:
# Convert Over_Value to float so we can multiply

df['Over_Value'] = df['Over_Value'].astype(float)

# Define a function to compute the odds based on the sign
def compute_odds(row, column):
    value = float(row[column + '_Value_Split'])
    odds = row[column + '_Odds']
    
    if odds.startswith('-'):
        return abs(value) / (abs(value) + 100)
    elif odds.startswith('+'):
        return 100 / (int(value) + 100)
    else:
        return None

# Apply the function to compute the odds for 'Over' and 'Under'
df['Over_Odds_Computed'] = df.apply(lambda row: compute_odds(row, 'Over'), axis=1)
df['Under_Odds_Computed'] = df.apply(lambda row: compute_odds(row, 'Under'), axis=1)
df['Final_Converted'] = (df['Over_Odds_Computed']+0.5)*df['Over_Value']
# Display the updated dataframe
df.head(25)

,Team,Over,Under,Over_Value,Over_Odds,Under_Value,Under_Odds,Over_Value_Split,Over_Sign,Under_Value_Split,Under_Sign,Over_Odds_Computed,Under_Odds_Computed,Final_Converted
0,Air Force,8.5 +125,8.5 −150,8.5,+125,8.5,-150,125.0,+,150.0,-,0.444444,0.600000,8.027778
1,Akron,3.5 −120,3.5 −105,3.5,-120,3.5,-105,120.0,-,105.0,-,0.545455,0.512195,3.659091
2,Alabama,10.5 +135,10.5 −155,10.5,+135,10.5,-155,135.0,+,155.0,-,0.425532,0.607843,9.718085
3,Appalachian State,6.5 −110,6.5 −115,6.5,-110,6.5,-115,110.0,-,115.0,-,0.523810,0.534884,6.654762
4,Arizona,5 −120,5 +100,5.0,-120,5,+100,120.0,-,100.0,+,0.545455,0.500000,5.227273
5,Arizona State,5 +120,5 −140,5.0,+120,5,-140,120.0,+,140.0,-,0.454545,0.583333,4.772727
6,Arkansas,7 +100,7 −120,7.0,+100,7,-120,100.0,+,120.0,-,0.500000,0.545455,7.000000
7,Arkansas State,4.5 −145,4.5 +120,4.5,-145,4.5,+120,145.0,-,120.0,+,0.591837,0.454545,4.913265
8,Army,6 +105,6 −130,6.0,+105,6,-130,105.0,+,130.0,-,0.487805,0.565217,5.926829
9,Auburn,6.5 −135,6.5 +115,6.5,-135,6.5,+115,135.0,-,115.0,+,0.574468,0.465116,6.984043


And everything looks good. I like to hang onto any of our separate columns to be able to do Quality Assurance testing on the separate columns as there's multiple steps to our calculations.

In [45]:
df.drop(['Under_Value', 'Under_Odds', 'Over_Odds_Computed', 'Under_Odds_Computed', 'Over_Value_Split',
         'Over_Sign', 'Under_Value_Split', 'Under_Sign','Over_Odds'], axis =1, inplace=True)

In [46]:
df.sort_values(by='Final_Converted', ascending=False, ignore_index=True).head(30)

,Team,Over,Under,Over_Value,Final_Converted
0,Georgia,11.5 +130,11.5 −150,11.5,10.750000
1,USC,9.5 −165,9.5 +140,9.5,10.665094
2,Michigan,10.5 −105,10.5 −115,10.5,10.628049
3,Ohio State,10.5 −105,10.5 −115,10.5,10.628049
4,Clemson,9.5 −140,9.5 +120,9.5,10.291667
5,Florida State,10 −105,10 −115,10.0,10.121951
6,Penn State,9.5 −130,9.5 +110,9.5,10.119565
7,Oklahoma,9.5 −115,9.5 −105,9.5,9.831395
8,Oklahoma State,9.5 −115,9.5 −105,9.5,9.831395
9,Texas,9.5 −115,9.5 −105,9.5,9.831395


In [47]:
df.sort_values(by='Final_Converted', ascending=False, ignore_index=True).head(30).describe()

,Over_Value,Final_Converted
count,30.000000,30.000000
mean,9.116667,9.214688
std,0.979825,0.873582
min,7.500000,8.125000
25%,8.500000,8.397727
50%,9.500000,9.020202
75%,9.500000,9.831395
max,11.500000,10.750000


In [48]:
# Save to a CSV for our future usage

df.to_csv('CFB_23_Win_Totals.csv')

# In Conclusion

So in conclusion, we scraped some table data from DraftKings in order to manipulate that data to show which teams are projected to have the highest win totals with all things being equal. We had to separate the odds from the win total, and then use those odds to calculate the implied probabilty and added 50% to attempt to centralize the odds as much as possible. After that, we folded back in our win total to give us an apples to apples comparision of the data. Looks like after our conversion, our mean went up slightly and the std ticked down, showing that our centralizing process did smooth out some of the bumps affiliated with different odds for different win totals. This could also be from Georgia's win total going from above 2 standard deviations to "merely" 1.8 -- a modest improvement but an improvement nevertheless.

This CSV will be available for download if anyone else wants a proper 'anchor point' for the NCAAF season. An example of this is if a team in the top 25, like North Carolina or Kansas State, starts off 1-2 and plays a team in the fourth week as a sizable favorite, you'll have better insight where that number came from.

Also, keep in mind these odds will change as more money comes onto a specific side, the bookmakers will adjust their number to mitigate overleveraging a position. This could be informative itself as the more the odds change the more action that position is taking (assuming no new information regarding the team). Whether that's the public reading a news article about how Caleb Williams will dominate this season, or seasoned professionals moving markets, the odds movement is important to keep in mind when wagering this upcoming season.

Happy handicapping -- September cannot come soon enough.

# PS

For those that stuck around, we need to come back around and see which teams we left off the ticket. Here, we can double check our results against another betting site, reviewjournal, and do a join to see which teams were excluded from DraftKings totals. The two tables could be the same which would tell me that there's a reason like an injury or coaching issue that the book would wait out prior to posting a number.

In [49]:
url = 'https://www.reviewjournal.com/sports/betting/college-football-regular-season-win-totals-full-list-of-133-teams-2785339/'

page = requests.get(url)
  
soup = BeautifulSoup(page.text, 'html.parser')

soup


<!DOCTYPE html>

<html class="nojs" lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<title>NCAA College football regular season win totals — FULL LIST of 133 teams | Las Vegas Review-Journal</title>
<meta content="max-image-preview:large" name="robots">
<link href="//cdnjs.cloudflare.com" rel="dns-prefetch">
<link href="//securepubads.g.doubleclick.net" rel="dns-prefetch"/>
<link href="//projects.reviewjournal.com" rel="dns-prefetch"/>
<link href="//stats.wp.com" rel="dns-prefetch"/>
<link href="//fonts.googleapis.com" rel="dns-prefetch"/>
<link href="//v0.wordpress.com" rel="dns-prefetch"/>
<link href="https://www.reviewjournal.com/feed/" rel="alternate" title="Las Vegas Review-Journal » Feed" type="application/rss+xml"/>
<link href="https://www.reviewjournal.com/comments/feed/" rel="alternate" title="Las Vegas Review-Journal » Comments Feed" type=

In [51]:
import re

# Trimming the beginning and ending of our dataset because we know it's not important
body = soup.findAll('p')[2:-12]

# Converting our list to a string so we can use "findall and regular expressions
data = str(body)
team_names = re.findall(r'<p>([A-Z][\w\s]+)', data)
win_totals = re.findall(r'; ([\d½]+)', data)
odds = re.findall(r'\((u-\d+|\bo-\d+)\)', data)

# Putting this info into a dataframe
df_133 = pd.DataFrame({
    'Team': team_names,
    'Win Total': win_totals,
#    'Odds': odds
})

df_133.head(25)

,Team,Win Total
0,Georgia,11
1,Michigan,10½
2,Ohio State,10½
3,USC,10
4,Clemson,10
5,Alabama,10
6,Florida State,9½
7,Texas,9½
8,Penn State,9½
9,LSU,9½


In [52]:
# Ensure we have 133 teams

df_133.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Team       133 non-null    object
 1   Win Total  133 non-null    object
dtypes: object(2)
memory usage: 2.2+ KB


In [53]:
# Perform left merge and indicate the source of each row
merged = pd.merge(df_133, df, on='Team', how='left', indicator=True, sort = True)

# Filter rows that exist in the first dataset but not in the second dataset
rows_only_in_df_133 = merged[merged['_merge'] == 'left_only']

print("this is the amount of differenec : ", len(rows_only_in_df_133),rows_only_in_df_133)

this is the amount of differenec :  24                        Team Win Total Over Under  Over_Value  Final_Converted  \
16            Brigham Young        5½  NaN   NaN         NaN              NaN   
17                  Buffalo        6½  NaN   NaN         NaN              NaN   
19          Central Florida         7  NaN   NaN         NaN              NaN   
27              Connecticut        4½  NaN   NaN         NaN              NaN   
32         Florida Atlantic        7½  NaN   NaN         NaN              NaN   
33    Florida International        3½  NaN   NaN         NaN              NaN   
40                   Hawaii        3½  NaN   NaN         NaN              NaN   
46            Jackson State        5½  NaN   NaN         NaN              NaN   
59                  Memphis        7½  NaN   NaN         NaN              NaN   
60                   Miami         7½  NaN   NaN         NaN              NaN   
61                   Miami         6½  NaN   NaN         NaN          

So there's 18 rows here we must calculate that are redundant or not. Another issue I thought of is different tables using different team names or abbreviations. We will not tackle a suitable solution for this project, but we will use the merge similar to above to list all the teams in our original DataFrame that doesn't show up, and then manually check those lists to find some missing links.

In [54]:
# Perform left merge and indicate the source of each row
merged = pd.merge(df, df_133, on='Team', how='left', indicator=True)

# Filter rows that exist in the first dataset but not in the second dataset
rows_only_in_df = merged[merged['_merge'] == 'left_only']

print("this is the amount of difference : ", len(rows_only_in_df), rows_only_in_df)

this is the amount of difference :  21                             Team      Over     Under  Over_Value  \
16                           BYU  5.5 −120  5.5 +100         5.5   
28                           FIU  2.5 −145  2.5 +120         2.5   
36             Hawaii (13 Games)  3.5 +115  3.5 −140         3.5   
42            Jacksonville State  5.5 −110  5.5 −115         5.5   
50           Louisiana-Lafayette  5.5 −110  5.5 −115         5.5   
56                      Miami FL  7.5 +120  7.5 −140         7.5   
57                      Miami OH  6.5 −110  6.5 −115         6.5   
60              Middle Tennessee  6.5 −125  6.5 +100         6.5   
66                        Nevada    3 −160    3 +135         3.0   
68   New Mexico State (13 Games)    5 −120    5 −105         5.0   
80                      Ole Miss  6.5 +100  6.5 −120         6.5   
87                   Sam Houston    4 −125    4 +100         4.0   
90                           SMU    8 −110    8 −115         8.0   
94       

The teams our dataset is missing in alphabetical order are Buffalo, FAU (go owls), Memphis, Toledo, and Rutgers. I can't seem to find the last missing link! If someone does find it please let me know!

I kept this part in to demonstrate that even though these tasks are automated by very powerful machines, if one doesn't perform Quality Assurance testing they could get themselves in trouble for ommiting data. Not only that, but the process to find the missing data or issue could be painstaking and tedious, as we had to manuelly check the differences between the 2 separate datasets above. Sometimes the code writes itself, other times, it requires attention to detail but all of it's necessary to try and find some truth in the matter.

For the sake of this project, I will take the one missing link for the 132 we were able to process and compute.

Thanks again for reading!